In [ ]:
from google.colab import drive
drive.mount('/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive/


In [ ]:
import numpy as np

X = np.load("/drive/My Drive/ML Data/Grey_Image_Filtered.npy")
print(X.shape)

(23708, 20000)


In [ ]:
Y = np.load("/drive/My Drive/ML Data/Age.npy")
print(Y.shape)

(23708, 1)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
Y = Y.astype(np.uint32)
y = Y.ravel()

In [ ]:
scaler1 = StandardScaler()
scaler1.fit(X)
x_std = scaler1.transform(X)

print("Done")

Done


In [ ]:
scaling = MinMaxScaler(feature_range=(-1,1)).fit(x_std)
x_std = scaling.transform(x_std)

print(x_std[1])

[-1.     1.     1.    ... -0.938 -0.938 -0.874]


In [ ]:
del X

print(x_std.shape)

(23708, 20000)


In [ ]:
y_final = []
x_final = []

for i in range(0,23708):
  if(Y[i] > 14 and Y[i]< 26):
    y_final.append(Y[i])
    x_final.append(x_std[i]) 


In [ ]:
y_final = np.asarray(y_final)
x_final = np.asarray(x_final)
print(y_final.shape, type(y_final))
print(x_final.shape, type(x_final))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(x_final,y_final,test_size=0.2, random_state=30)
del x_std


In [ ]:
pca = PCA(n_components=0.95, svd_solver='full')

X_train_final = pca.fit_transform(X_train)
X_test_final = pca.transform(X_test)

print("Done with PCA\n") 

Done with PCA



In [ ]:
y_test = y_test.ravel()
y_train = y_train.ravel()

In [ ]:
# Linear model
svm_l = SVR(kernel = 'linear')

parameters_l = {'C':[0.001, 0.1, 100, 10e5]}
grid_l = GridSearchCV(svm_l, parameters_l, cv=5)
grid_l.fit(X_train, y_train)
y_pred_l = grid_l.predict(X_test)
acc_l = accuracy_score(y_test,y_pred_l)
print('Test Accuracy Score for linear kernel: ' + str(acc_l))

print("Done with Linear model")             # Just an interrupt



In [ ]:
y_test = y_test.astype(np.float)

In [ ]:
from sklearn.metrics import mean_absolute_error as mae

acc_l = mae(y_test,y_pred_l)
print('Test Accuracy Score for linear kernel: ' + str(acc_l))
print("Done with Linear model")             # Just an interrupt


In [ ]:
# Sigmoid Model
svm_s = SVR(kernel='sigmoid')

parameters_s = {'C':[0.001, 0.1, 100, 10e5], 'gamma':[10,1,0.1,0.01]}
grid_s = GridSearchCV(svm_s, parameters_s, cv=5)
grid_s.fit(X_train, y_train)
y_pred_s = grid_s.predict(X_test)
acc_s = mae(y_test,y_pred_s)
print('Test Accuracy Score for sigmoid kernel : ' + str(acc_s))

print("Done with Sigmoid model")             # Just an interrupt



In [ ]:
import joblib

In [ ]:
filename1 = '/drive/My Drive/ML Data/Grey_Age_UTK_Sobel_PCA/LinearModel.sav'
joblib.dump(grid_l, filename1)

filename2 = '/drive/My Drive/ML Data/Grey_Age_UTK_Sobel_PCA/SigmoidModel.sav'
joblib.dump(grid_s, filename2)

['/drive/My Drive/ML Data/Grey_Age_UTK_Sobel_PCA/SigmoidModel.sav']

In [ ]:
# Polynomial model

svm_poly = SVR(kernel = 'poly')

parameters_poly = {'C':[ 0.1, 100], 'gamma':[1,0.01]}
grid_poly = GridSearchCV(svm_poly, parameters_poly, cv=5)
grid_poly.fit(X_train, y_train)
y_pred = grid_poly.predict(X_test)
acc_poly = mae(y_test,y_pred_poly)
print('Test Accuracy Score for polynomial kenel: ' + str(acc_poly))


In [ ]:
filename4 = '/drive/My Drive/ML Data/Grey_Age_UTK_Sobel_PCA/PolynomialModel.sav'
joblib.dump(grid_poly, filename4)

In [ ]:
# rbf Model
svm_rbf = SVR(kernel = 'rbf')

parameters_rbf = {'C':[0.001, 0.1, 100, 10e5], 'gamma':[10,1,0.1,0.01]}
grid_rbf = GridSearchCV(svm_rbf, parameters_rbf, cv=5)
grid_rbf.fit(X_train, y_train)
y_pred_rbf = grid_rbf.predict(X_test)
acc_rbf = mae(y_test,y_pred_rbf)
print('Test Accuracy Score for rbf kernel: ' + str(acc_rbf))

print("Done with rbf model")             # Just an interrupt


In [ ]:
 print(X_test.shape, X_train.shape)
 print(y_test.dtype, y_pred_l.dtype)

(798, 20000) (3188, 20000)
uint32 float64


In [ ]:
filename3 = '/drive/My Drive/ML Data/Grey_Age_UTK_Sobel_PCA/RBFModel.sav'
joblib.dump(grid_rbf, filename3)



In [ ]:
# print best parameter after tuning 
print('Linear',grid_l.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print('Linear',grid_l.best_estimator_)


# print best parameter after tuning 
print('Sigmoid', grid_s.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print('Sigmoid',grid_s.best_estimator_) 


# print best parameter after tuning 
print('RBF',grid_rbf.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print('RBF',grid_rbf.best_estimator_) 



# print best parameter after tuning 
print('POLY', grid_poly.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print('POLY',grid_poly.best_estimator_) 